In [1]:
import cv2

In [2]:
import math

def calculateCartesianCoordinates(radius, azimuth):
    return radius * math.sin(math.radians(azimuth)), radius * math.cos(math.radians(azimuth))

In [3]:
from __future__ import print_function
import yaml
try:
    from yaml import CLoader as Loader
except ImportError:
    from yaml import Loader

import gzip
import os.path
import numpy as np

# Reading yml.gz data

# Сhange this to where data is stored
data_dir = '/Users/mihhaildunajev/projects/Diploma/radar_calib_data'


def ungzip(yaml_in_directory):
    # наши ямлы требуют небольшой ректификации перед использованием
    ungzipped = gzip.open(yaml_in_directory, 'rt')
    ungzipped.readline()
    ungzipped = ungzipped.read()
    ungzipped = ungzipped.replace(':', ': ')

    # собственно парсинг
    yml = yaml.load(ungzipped, Loader=Loader)
    return yml

def read_image_grabmsecs(yml_path):
    yml_data = ungzip(yml_path)
    image_frames = [sh['leftImage']
                    for sh in yml_data['shots'] if 'leftImage' in sh.keys()]
    
    data = np.zeros(shape=(len(image_frames), 1), dtype=int)
    i_real = 0
    for i_fr in image_frames:
        data[i_real, 0] = int(i_fr['grabMsec'])
        i_real += 1

    data = data[:i_real, :]
    return data

grabmsecs = read_image_grabmsecs(os.path.join(data_dir, 't24.305.028.info.yml.gz')).flatten()

In [4]:
import yaml
import numpy as np

# Reading yml data

# Сhange this to where data is stored
camera_path = "/Users/mihhaildunajev/projects/Diploma/radar_calib_data/calibs/camera.yml"

try:
    from yaml import CLoader as Loader
except ImportError:
    from yaml import Loader
    
def read_yml(yml_path):
    yml = yaml.load(yml_path, Loader=Loader)
    return yml

def opencv_matrix(loader, node):
    mapping = loader.construct_mapping(node, deep=True)
    dt = np.dtype(mapping["dt"])
    mat = np.array(mapping["data"], dt)
    mat.resize(mapping["rows"], mapping["cols"])
    return mat

stream = open(camera_path, 'r')
stream.readline()
stream = stream.read()
stream = stream.replace(':', ': ')
yaml.add_constructor(u"tag:yaml.org,2002:opencv-matrix", opencv_matrix)
data = yaml.load(stream)


In [5]:
# Creating Camera object for central projection 

class Camera:
    '''Pinhole camera projection'''
    def __init__(self, r, t, K, size):
        if  not (t.shape == (3,) and r.shape == (3,) and K.shape == (3,3)):
            raise ValueError('bad constructor args')
        self.r = r
        self.t = t
        self.K = K
        self.size = size
        R = rot_matrix(r)
        self.P = K.dot(np.hstack([R, t.reshape(t.shape[0], 1)]))
        pass
    
    def translate(self, points, t):
        return np.array([point + t for point in points])
    
    def rotate(self, points, rot_mat):
        return np.array([rot_mat.dot(point) for point in points])
    
    def transform(self, points):
        return self.rotate(points, rot_matrix([np.pi / 2.0, 0, 0]))
    
    def project(self, points):
        if points.shape[1] == 3:
            R = rot_matrix(r)
            translated = self.translate(points, self.t)
#             print("Translated: ", translated)
            rotated = self.rotate(translated, R.T)
#             print("Rotated: ", rotated)
            tranformed = self.transform(rotated)
#             print("Transformed: ", tranformed)
            homo_pts = self.K.dot(tranformed.T).T
#             print("homo_pts: ", homo_pts)
        elif points.shape[1] == 4:
            homo_pts = self.P.dot(points.T)
        else:
            raise ValueError('Incorrect points size for Camera.project: %s' % \
                             str(points.shape))
        return homo_pts[:, : 2] / homo_pts[:, 2].reshape(-1, 1)
    
def rot_matrix(angles):
    cos = [np.math.cos(a) for a in angles]
    sin = [np.math.sin(a) for a in angles]
    Rz = np.array([[cos[2], -sin[2], 0],
                   [sin[2], cos[2],  0],
                   [     0,      0,  1]])

    Ry = np.array([[ cos[1], 0, sin[1]],
                   [      0, 1,      0],
                   [-sin[1], 0, cos[1]]])

    Rx = np.array([[1,      0,       0],
                   [0, cos[0], -sin[0]],
                   [0, sin[0], cos[0]]])
    #Rs = [Rx, Ry, Rz]
    return Ry.dot(Rx).dot(Rz)
    
K = data["K"]
r = data["r"].flatten()
t = data["t"].flatten()
img_size = (data["sz"][0], data["sz"][1])
print('K = ', K)
print('r = ', r)
print('t = ', t)
print('Image size = ', img_size)
cam = Camera(r=r, t=-t, K=K, size=img_size)

K =  [[1.39022723e+03 0.00000000e+00 9.62339056e+02]
 [0.00000000e+00 1.39022723e+03 5.31100969e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
r =  [ 0.04163906 -0.02612186 -0.02203633]
t =  [ 0.15882312 -1.58        1.25712767]
Image size =  (1920, 1080)


In [6]:
from __future__ import print_function

import pandas as pd
import os
import os.path

# Change this to where data is stored
data_dir = '/Users/mihhaildunajev/projects/Diploma/radar_calib_data/radar_detections' 

# Change this to where data is stored
video_dir = '/Users/mihhaildunajev/projects/Diploma/radar_calib_data'

# UI sliders callback functions

def changeTX(x):
    cam.t[0] = -(x - 10000)/1000.0
    
def changeTY(y):
    cam.t[1] = -(y - 10000)/1000.0
    
def changeTZ(z):
    cam.t[2] = -(z - 10000)/1000.0
    
def changeRX(x):
    cam.r[0] = (x - 10000)/1000.0/100.0
    
def changeRY(y):
    cam.r[1] = (y - 10000)/1000.0/100.0
    
def changeRZ(z):
    cam.r[2] = (z - 10000)/1000.0/100.0

def rescale_frame(frame, percent=75):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

# Reading a single tsv file by path
def read_table(path):
    try:
        return pd.read_csv(path, delimiter='\t', skiprows=[1])
    except pd.errors.EmptyDataError as ede:
        return []

# Reading all tsv files in a folder
def read_tables(folder):
    """reads all tables found in directory and concatenates them into one big
    table
    """
    common_table = pd.DataFrame()
    empty_files = []
    nparsed = 0

    for root, dirs, files in os.walk(folder):
        for f in [f for f in files if os.path.splitext(f)[-1] == '.tsv']:
            # print('scanning file', f, '...', end=' ')
            try:
                # Основной вызов в чтении. delimiter='\t', чтобы считать табы разделителями,
                # skiprows=[1], чтобы пропустить строку с типами данных.
                table = pd.read_csv(os.path.join(root, f), delimiter='\t', skiprows=[1])
            except pd.errors.EmptyDataError as ede:
                # print('Empty file!!')
                empty_files.append(f)
            finally:
                # print(table.shape)
                common_table = common_table.append(table)

            nparsed += 1

    print('parsed', nparsed, 'files. Rows: ', common_table.shape[0], ' empty files number:', len(empty_files))
    print('empty files list: [', ', '.join(empty_files), ']\n')
    
    # чтобы установить сквозную нумерацию во всех прочитанных таблицах
    common_table = common_table.reset_index(drop=True)
    
    return common_table

if __name__ == '__main__':
    cv2.destroyAllWindows()
#     table = read_table(os.path.join(data_dir, 't24.305.028.tsv')) # Reading t24.305.028.tsv radar points only
    table = read_tables(data_dir) # Reading all points
    selectedDataTable = table.iloc[:, [1, 3, 4]]
    uniqueDataTable = selectedDataTable.grabMsec.unique()
    cap = cv2.VideoCapture(os.path.join(video_dir, 't24.305.028.left.avi'))
    # Creating UI
    cv2.namedWindow('frame')
    cv2.createTrackbar('t_x', 'frame', 10159, 20000, changeTX)
    cv2.createTrackbar('t_y', 'frame', 8420, 20000, changeTY)
    cv2.createTrackbar('t_z', 'frame', 11257, 20000, changeTZ)
    cv2.createTrackbar('r_x', 'frame', 14164, 20000, changeRX)
    cv2.createTrackbar('r_y', 'frame', 7388, 20000, changeRY)
    cv2.createTrackbar('r_z', 'frame', 7797, 20000, changeRZ)
    # Current index of an element from yml.gz's grabmsec list
    grabmsecsIndex = 0
    # Current index of an element from radar tsv's grabmsec list
    radarDetectionsIndex = 0
    while(True):
        ret, frame = cap.read()
        if frame is None: # Loop video
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            grabmsecsIndex = 0
            radarDetectionsIndex = 0
            continue
        frame = rescale_frame(frame, percent=50)
        if grabmsecsIndex >= len(grabmsecs):
            # There are no more frames to show
            continue
            
        # Calculate the radar grabmsec index closest to the camera grabmsec index
        while radarDetectionsIndex+1 < len(uniqueDataTable) and uniqueDataTable[radarDetectionsIndex+1] < grabmsecs[grabmsecsIndex]:
            radarDetectionsIndex += 1
            
        if radarDetectionsIndex+1 >= len(uniqueDataTable):
            # There are no more radar detections to show. 
            grabmsecsIndex += 1
            # Show frame without any detections and continue
            cv2.imshow('frame', frame)
            continue
        # Get current grabmsecs of both camera and radar
        currentGrabmsec = grabmsecs[grabmsecsIndex]
        currentFrameDetections = selectedDataTable.loc[selectedDataTable['grabMsec'] == uniqueDataTable[radarDetectionsIndex]]
        
        # Draw all detections one by one
        for _, detection in currentFrameDetections.iterrows():
            x_r, y_r = calculateCartesianCoordinates(detection['range'], detection['azimuth'])
            z_r = 0 
            points = cam.project(np.array([np.array([x_r, y_r, z_r])]))
            x, y = int(points[0][0]*0.5), int(points[0][1]*0.5)
            
            if x < frame.shape[1] and y < frame.shape[0] and x > 0 and y > 0:
                cv2.circle(frame, (x, y), 5, (0,255,0), -1)
                
        # Show frame with detections
        cv2.imshow('frame', frame)
        
        # Go to the next camera grabmsec value
        grabmsecsIndex += 1
        
        # Checking if the user pressed q key to exit the video
        if cv2.waitKey(1) & 0xFF == ord('q'): # Press 'Q' key to exit
            break
            
    # Destroing the video window
    cv2.destroyAllWindows()

parsed 4 files. Rows:  21836  empty files number: 0
empty files list: [  ]

